In [1]:
from config import CONFIG
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
%matplotlib notebook
import math
import numpy as np
import pandas as pd
import sys
import time

In [46]:
path = CONFIG['PATH']
start_date='2017-10-15'
end_date='2017-11-15'
dates=pd.date_range(start_date, end_date)
merged_candlesticks=pd.DataFrame(index=dates)

for pair in CONFIG['PAIRS']:
    candlesticks = pd.read_csv(path + "/data/"+ pair.replace('/','-') +"/candlesticks.csv", parse_dates=True, usecols=['date','close'], na_values=['nan'])
    candlesticks['date'] = pd.Series([dt.datetime.fromtimestamp(int(ts)).strftime('%Y-%m-%d')
                                     for ts in candlesticks['date']]).values
    candlesticks = candlesticks.set_index('date')
    candlesticks = candlesticks.rename(columns={'close':pair})
    merged_candlesticks=merged_candlesticks.join(candlesticks)
print(merged_candlesticks) 

             ETH/BTC   LTC/BTC   BTS/BTC  STEEM/BTC
2017-10-15  0.057951  0.011236  0.000010   0.000182
2017-10-16  0.056342  0.010569  0.000010   0.000188
2017-10-17  0.056180  0.010887  0.000010   0.000190
2017-10-18  0.053880  0.010460  0.000009   0.000184
2017-10-19  0.050510  0.010013  0.000008   0.000167
2017-10-20  0.049900  0.009660  0.000009   0.000162
2017-10-21  0.049230  0.009419  0.000009   0.000161
2017-10-22  0.048190  0.009245  0.000008   0.000160
2017-10-23  0.053761  0.010094  0.000010   0.000181
2017-10-24  0.051703  0.009800  0.000009   0.000176
2017-10-25  0.050102  0.009409  0.000009   0.000163
2017-10-26  0.051487  0.009537  0.000009   0.000172
2017-10-27  0.051291  0.009459  0.000009   0.000164
2017-10-28  0.049330  0.009221  0.000009   0.000160
2017-10-29  0.050030  0.009164  0.000009   0.000157
2017-10-30  0.046981  0.008576  0.000011   0.000147
2017-10-31  0.042702  0.007823  0.000009   0.000132
2017-11-01  0.040600  0.007711  0.000008   0.000127
2017-11-02  

In [ ]:
window = 20
no_of_std = 2

rm_btc = pd.Series(candlesticks['close']).rolling(window=window).mean()
rstd_btc = pd.Series(candlesticks['close']).rolling(window=window).std()
candlesticks['Rolling Mean'] = rm_btc
candlesticks['Rolling Mean'].fillna(method='bfill', inplace=True)


candlesticks['Bollinger High'] = rm_btc + (rstd_btc * no_of_std)
candlesticks['Bollinger High'].fillna(method='bfill', inplace=True)

candlesticks['Bollinger Low'] = rm_btc - (rstd_btc * no_of_std)
candlesticks['Bollinger Low'].fillna(method='bfill', inplace=True)

x = candlesticks['date']
ax = candlesticks[['close','Rolling Mean', 'Bollinger High', 'Bollinger Low']].plot(x, title="BTC/ETC Bollinger Bands", label="BTC/ETC")

ax.set_xlabel("Date")
ax.set_ylabel("Price")
ax.legend(loc='upper left')